In [1]:
%load_ext autoreload
%autoreload 2
from palmari.processing import *
from palmari import Experiment, TifPipelineWidget
from palmari.processing.steps.quot_localizer import MaxLikelihoodLocalizer
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

# Your first Experiment

In [2]:
exp = Experiment(data_folder="/Users/hverdier/palm_tools_data/DATA_FOLDER",
    export_folder="/Users/hverdier/palm_tools_data/EXPORT_FOLDER")

In [3]:
# Indicate conditions in the index_df table of the Experiment
# This can be done automatically if you create a subclass of Experiment (see below)
exp.index_df["condition"] = "A"
exp.index_df.loc[exp.index_df.file.str.contains("B"),"condition"] = "B"

In [4]:
exp.index_df

,file,ID,condition
0,CONDITION A/Experience1/CellZone1/ROI.tif,4c673d7a3babf12f,A
1,CONDITION A/Experience1/CellZone0/ROI.tif,0a4f0c4d5a770f14,A
2,CONDITION A/Experience0/CellZone2/ROI.tif,1aeb07e6d906fa6d,A
3,CONDITION A/Experience0/CellZone1/ROI.tif,272126b5e72b50a4,A
4,CONDITION A/Experience0/CellZone0/ROI#0.tif,e050273c2b28d429,A
5,CONDITION B/Exp0/cellZone2/ROI#0.tif,448d0bb5d719692d,B
6,CONDITION B/Exp0/cellZone1/ROI.tif,22ec7db7a6a63186,B
7,CONDITION B/Exp0/cellZone0/ROI.tif,2cf987b2e99bb1ed,B
8,CONDITION B/Exp1/cellZone2/ROI.tif,0628d1f1d8516a68,B
9,CONDITION B/Exp1/cellZone1/ROI.tif,662132ba6f8b14f0,B


# Make your Experiment subclass

In [5]:
class MyExp(Experiment):

    @property
    def custom_fields(self):
        return {"cell_zone":-2,"exp":-3,"condition":-4}

In [6]:
exp = MyExp(data_folder="/Users/hverdier/palm_tools_data/DATA_FOLDER",
    export_folder="/Users/hverdier/palm_tools_data/EXPORT_FOLDER")

In [7]:
exp.index_df

,file,ID,cell_zone,exp,condition
0,CONDITION A/Experience1/CellZone1/ROI.tif,4c673d7a3babf12f,CellZone1,Experience1,CONDITION A
1,CONDITION A/Experience1/CellZone0/ROI.tif,0a4f0c4d5a770f14,CellZone0,Experience1,CONDITION A
2,CONDITION A/Experience0/CellZone2/ROI.tif,1aeb07e6d906fa6d,CellZone2,Experience0,CONDITION A
3,CONDITION A/Experience0/CellZone1/ROI.tif,272126b5e72b50a4,CellZone1,Experience0,CONDITION A
4,CONDITION A/Experience0/CellZone0/ROI#0.tif,e050273c2b28d429,CellZone0,Experience0,CONDITION A
5,CONDITION B/Exp0/cellZone2/ROI#0.tif,448d0bb5d719692d,cellZone2,Exp0,CONDITION B
6,CONDITION B/Exp0/cellZone1/ROI.tif,22ec7db7a6a63186,cellZone1,Exp0,CONDITION B
7,CONDITION B/Exp0/cellZone0/ROI.tif,2cf987b2e99bb1ed,cellZone0,Exp0,CONDITION B
8,CONDITION B/Exp1/cellZone2/ROI.tif,0628d1f1d8516a68,cellZone2,Exp1,CONDITION B
9,CONDITION B/Exp1/cellZone1/ROI.tif,662132ba6f8b14f0,cellZone1,Exp1,CONDITION B


# Create pipelines

## Default

In [8]:
tp = TifPipeline.from_dict({"name":"test","localizer":{"MaxLikelihoodLocalizer":{}},"tracker":{"DiffusionTracker":{}}})
print(tp)

TIF Processing pipeline
-----------------------
Movie preprocessing steps :
-----------------------
Localizer :
	 MTT (quot) localizer (MaxLikelihoodLocalizer) :
		 detect_params : {'method': 'llr', 'k': 1.5, 'w': 21, 't': 16.0}
		 loc_params : {'method': 'ls_int_gaussian', 'window_size': 9, 'sigma': 1.5, 'ridge': 0.0001, 'convergence': 0.0001, 'divergence': 1.0, 'max_iter': 10, 'damp': 0.3, 'camera_bg': 0.0, 'camera_gain': 1.0}

-----------------------
Localization processing steps :
-----------------------
Tracker :
	 MTT tracker (Trackpy) (DiffusionTracker) :
		 params : {}




## Export and load pipelines

In [9]:
tp.to_yaml("/Users/hverdier/palm_tools_data/pipeline_test.yaml")
tp = TifPipeline.from_yaml("/Users/hverdier/palm_tools_data/pipeline_test.yaml")

# Run the pipeline on an Acquisition

In [10]:
# Access files of your experiment as if the Experiment instance was a list
exp[1]

'CONDITION A/Experience1/CellZone0/ROI.tif'

In [11]:
# To start, we'll process just one file. A movie is represented by an Acquisition
acq = Acquisition(exp[2],experiment=exp, tif_pipeline=tp)

In [13]:
tp.process(acq,force_reprocess=True)

[########################################] | 100% Completed |  6min 46.0s


AttributeError: 'DiffusionTracker' object has no attribute 'max_diffusivity'

In [31]:
# That's it, you've got your localizations !
acq.locs.head()

,y,x,I0,bg,y_err,x_err,I0_err,bg_err,H_det,error_flag,snr,rmse,n_iter,y_detect,x_detect,frame,t,n
0,6.582300,4.405658,2588.156773,229.481235,0.193045,0.193101,94.739182,5.002356,295.326456,0,324.389658,43.846684,10.0,68.0,45.0,0,0.00,0
1,4.120426,5.696877,1214.116699,218.691789,0.174472,0.174410,77.895864,2.255731,23.471315,0,116.123533,18.398970,10.0,43.0,59.0,1,0.03,1
2,6.570093,4.453834,1640.431434,229.267773,0.194981,0.194963,88.868977,3.367126,115.839402,0,168.622322,28.687795,10.0,68.0,46.0,2,0.06,2
3,7.071795,3.589543,5456.870991,175.653199,0.178903,0.179397,98.469162,9.380361,864.231395,0,1123.999118,84.083830,10.0,73.0,36.0,2,0.06,3
4,1.421944,4.353109,805.817099,200.564218,0.273095,0.272910,80.363217,2.430550,177.180313,0,46.385226,20.006585,10.0,14.0,45.0,3,0.09,4


In [36]:
# They're stored here
acq.locs_path

'/Users/hverdier/palm_tools_data/EXPORT_FOLDER/test/CONDITION A/Experience1/CellZone0/ROI.locs'

In [33]:
acq.locs[["x","y"]] = acq.locs[["y","x"]]

In [37]:
# See localizations and tracks overlayed on the movie in Napari
acq.view(min_traj_length=5)

/Users/hverdier/opt/miniconda3/envs/gratin/lib/python3.8/site-packages/numpy/core/numeric.py:2446: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  return bool(asarray(a1 == a2).all())
/Users/hverdier/opt/miniconda3/envs/gratin/lib/python3.8/site-packages/numpy/core/numeric.py:2446: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  return bool(asarray(a1 == a2).all())
/Users/hverdier/opt/miniconda3/envs/gratin/lib/python3.8/site-packages/numpy/core/numeric.py:2446: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  return bool(asarray(a1 == a2).all())


In [30]:
# Visualize the effect of each of the pipeline's steps here :
TifPipelineWidget.view_pipeline(tp,acq=acq)

# Run pipeline on the whole experiment !

In [9]:
tp.process(exp)